In [16]:
import os
import sys
import pandas as pd

In [17]:
test_df = pd.read_csv('../../data/german/test.csv')
train_df = pd.read_csv('../../data/german/train.csv')
val_df = pd.read_csv('../../data/german/valid.csv')


test_df.head()

,text,label,source
0,@user korrekt! Verstehe sowas nicht...,negative,sb_10k
1,Einparken können die Aliens auch nicht! #schlefaz,neutral,sb_10k
2,Der Dubbletimepart von Julien war ja mal sowas...,positive,sb_10k
3,#Instachallenge #Day16 #what #i #am #reading #...,negative,sb_10k
4,Also gleich. Mach noch das Video fertig.,neutral,sb_10k


In [18]:
# from sklearn.model_selection import train_test_split

# train_df, _ = train_test_split(
#     train_df,
#     test_size = 0.6 ,
#     stratify = train_df['label'],
#     random_state = 42
# )

In [19]:
train_df = train_df[train_df["label"] != "neutral"]
test_df  = test_df[test_df["label"]  != "neutral"]
val_df   = val_df[val_df["label"]   != "neutral"]

mapping = {'negative': 0, 'positive': 1}

# Create a new column with mapped values
train_df['sentiment_num'] = train_df['label'].map(mapping)
test_df['sentiment_num'] = test_df['label'].map(mapping)
val_df['sentiment_num'] = val_df['label'].map(mapping)

In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW
from datasets import Dataset

model_namtoktoktoksakdjajdsakldjaslkfnaskljdalskdhnalkdnsakldasdlkjsandklsandklsandlksasssasdsadqase-multilingual-uncased"
tok = BertTokenizer.from_pretrained(model_name)

MAX_LEN = 100  # tweets are short
#Converting to tensors
def tokenize(batch):
    return tok(
        batch["text"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_encodings = tokenize(train_df)
val_encodings = tokenize(val_df)
test_encodings = tokenize(test_df)


/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SentimentDataset(train_encodings, train_df["sentiment_num"].values)
val_dataset = SentimentDataset(val_encodings, val_df["sentiment_num"].values)
test_dataset = SentimentDataset(test_encodings, val_df["sentiment_num"].values)

In [22]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = 8)
#change number of labels based on keep/drop neutral class
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 33%|███▎      | 77/231 [00:50<01:33,  1.65it/s]

{'loss': 0.6477, 'grad_norm': 9.448026657104492, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


                                                
 33%|███▎      | 77/231 [00:52<01:33,  1.65it/s]

{'eval_loss': 0.49465617537498474, 'eval_runtime': 2.3106, 'eval_samples_per_second': 93.481, 'eval_steps_per_second': 6.059, 'epoch': 1.0}


 67%|██████▋   | 154/231 [01:47<00:46,  1.64it/s]

...

                                                 
100%|██████████| 231/231 [02:49<00:00,  1.53it/s]

{'eval_loss': 0.48722895979881287, 'eval_runtime': 2.2846, 'eval_samples_per_second': 94.546, 'eval_steps_per_second': 6.128, 'epoch': 3.0}


100%|██████████| 231/231 [02:51<00:00,  1.35it/s]

{'train_runtime': 171.0498, 'train_samples_per_second': 21.503, 'train_steps_per_second': 1.35, 'train_loss': 0.46284836608094054, 'epoch': 3.0}


TrainOutput(global_step=231, training_loss=0.46284836608094054, metrics={'train_runtime': 171.0498, 'train_samples_per_second': 21.503, 'train_steps_per_second': 1.35, 'total_flos': 189008293284000.0, 'train_loss': 0.46284836608094054, 'epoch': 3.0})

In [23]:
metrics = trainer.evaluate()
print(metrics)

100%|██████████| 14/14 [00:02<00:00,  6.19it/s]

{'eval_loss': 0.48722895979881287, 'eval_runtime': 2.3023, 'eval_samples_per_second': 93.82, 'eval_steps_per_second': 6.081, 'epoch': 3.0}


In [24]:
predictions = trainer.predict(val_dataset)

100%|██████████| 14/14 [00:02<00:00,  6.24it/s]


In [25]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, classification_report

logits = predictions.predictions      # or predictions[0]
labels = predictions.label_ids        # or predictions[1]

y_pred = np.argmax(logits, axis=-1)

print("Accuracy:", accuracy_score(labels, y_pred))
print("F1 (macro):", f1_score(labels, y_pred, average="macro"))

print(
    classification_report(
        labels,
        y_pred,
        #target_names=["negative", "neutral", "positive"]
        target_names=["negative","positive"]
    )
)

Accuracy: 0.8101851851851852
F1 (macro): 0.8101485626085279
              precision    recall  f1-score   support

    negative       0.80      0.82      0.81       108
    positive       0.82      0.80      0.81       108

    accuracy                           0.81       216
   macro avg       0.81      0.81      0.81       216
weighted avg       0.81      0.81      0.81       216



Test Dataset

In [26]:
pred = trainer.predict(test_dataset)
y_true = test_df["sentiment_num"]
y_pred = np.argmax(pred.predictions, axis=1)


logits = pred.predictions   
labels = pred.label_ids   


print("Accuracy:", accuracy_score(labels, y_pred))
print("F1 (macro):", f1_score(labels, y_pred, average="macro"))

print(
    classification_report(
        labels,
        y_pred,
        #target_names=["negative", "neutral", "positive"]
        target_names=["negative","positive"]
    )
)

100%|██████████| 14/14 [00:02<00:00,  6.15it/s]

Accuracy: 0.7870370370370371
F1 (macro): 0.787018777330018
              precision    recall  f1-score   support

    negative       0.79      0.78      0.79       108
    positive       0.78      0.80      0.79       108

    accuracy                           0.79       216
   macro avg       0.79      0.79      0.79       216
weighted avg       0.79      0.79      0.79       216

